In [2]:
import numpy as np
import pandas as pd

In [3]:
import codecs
from io import open

In [4]:
import nltk
# nltk.download()
from nltk.sentiment.vader import SentimentIntensityAnalyzer

C:\Users\Raghav\Anaconda3\lib\site-packages\nltk\twitter\__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


## Pre - processing

In [60]:
data = pd.read_csv('train.csv')

In [193]:
def cleanData(csv_file, train = 1):
    
    data = pd.read_csv(csv_file)
    
    data = data.drop('replyToSID', 1)
    data = data.drop('truncated', 1)
    data = data.drop('screenName', 1)
    
    if train:
        data = data.drop('statusSource', 1)
        
    data = data.drop('favorited', 1)
    data = data.drop('replyToSN', 1)
    data = data.drop('replyToUID', 1)
    data = data.drop('isRetweet', 1)
    data = data.drop('retweeted', 1)
    data = data.drop('id', 1)
    data = data.drop('id.1', 1)
    data = data.drop('latitude', 1)
    data = data.drop('longitude', 1)
    data = data.drop('favoriteCount', 1)
    data = data.drop('retweetCount', 1)
    
    data.created = pd.to_datetime(data.created)
    
    data['textLength'] = data.text.apply(len)

    data['textSentiment'] = data.text.apply(lambda x: analyzer.polarity_scores(x)['compound'])
    
    data['createdHour'] = data.created.apply(lambda x: x.hour)
    data['createdDay'] = data.created.apply(lambda x: x.dayofweek)
    
    
    data = data.drop('text', 1)
    data = data.drop('created', 1)
    
    
    data.createdDay = pd.Categorical(data.createdDay)
    data.createdHour = pd.Categorical(data.createdHour)
    
    labels = []
    
    if train:
        labels = data.label
        data = data.drop('label', 1)
    
    
    return data, labels

## Train - test split

In [195]:
trainData, trainLabels = cleanData('train.csv', train = 1)

In [196]:
testData, testLabels = cleanData('test.csv', train = 0)

In [199]:
from sklearn.model_selection import train_test_split

In [105]:
from sklearn.model_selection import KFold

## Random Forest

In [104]:
from sklearn.ensemble import RandomForestClassifier

In [204]:
kf = KFold(n_splits=10)
trainData = np.array(trainData)

for i in range(1, 20):
    
    clfRF = RandomForestClassifier(n_estimators=i * 100, criterion="entropy")

    accuracy_lst = []

    for train_index, test_index in kf.split(trainData):

        data_train = trainData[train_index]
        data_test = trainData[test_index]
        labels_train = labels[train_index]
        labels_test = labels[test_index]

        clfRF.fit(data_train, labels_train)

        accuracy = np.sum(clfRF.predict(data_test) == labels_test) * 1.0/len(data_test)

        accuracy_lst.append(accuracy) 
        
    print i * 100, np.mean(accuracy_lst), np.std(accuracy_lst)

100 0.7171083927964662 0.06281410165092617
200 0.7088515120625213 0.05821494937441702
300 0.7070166496772001 0.05926520381313302
400 0.7134386680258239 0.062256789135299785
500 0.7134641522256201 0.061004516933901715
600 0.716199456337071 0.0628413755778691
700 0.7143645939517499 0.0583884395707241
800 0.7079425756031261 0.05514277370413377
900 0.7161909616038057 0.06865473695774138
1000 0.7070251444104655 0.05462376978545508
1100 0.7143560992184846 0.06434241155457426
1200 0.7134471627590895 0.061930323149715555
1300 0.7180343187223921 0.05949329425346044
1400 0.7116123003737683 0.06129255217988205
1500 0.713438668025824 0.05688788094624898
1600 0.7134471627590894 0.06069483113610427


KeyboardInterrupt: 

In [205]:
clfRF = RandomForestClassifier(n_estimators=1600, criterion="entropy")

In [206]:
clfRF.fit(trainData, trainLabels)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=1600, n_jobs=1, oob_score=False,
            random_state=None, verbose=0, warm_start=False)

In [207]:
clfRF_probs = clfRF.predict_proba(testData)

In [208]:
np.sum(np.max(clfRF_probs, axis = 1) < 0.6)

43

In [124]:
pd.DataFrame(clfRF.predict(testData)).to_csv('predictions_less_RF.csv', header=['Label'], index_label = ['ID'])

## SVM

In [125]:
from sklearn.svm import SVC

In [295]:
kf = KFold(n_splits = 10)
trainDataScaled = np.array(trainDataScaled)

for i in range(10, 20):
    
    for j in range(1, 20):
        
        clfSVM = SVC(C = i, gamma = j)

        accuracy_lst = []

        for train_index, test_index in kf.split(trainDataScaled):

            data_train = trainDataScaled[train_index]
            data_test = trainDataScaled[test_index]

            labels_train = labels[train_index]
            labels_test = labels[test_index]
            
            clfSVM.fit(data_train, labels_train) 

            accuracy = np.sum(clfSVM.predict(data_test) == labels_test) * 1.0/len(data_test)
            accuracy_lst.append(accuracy) 

        print i, j, np.mean(accuracy_lst), np.std(accuracy_lst)

10 1 0.7207696228338429 0.05179453430178334
10 2 0.7033299354400271 0.05285607346459183
10 3 0.7024379884471628 0.050721375134452434
10 4 0.6987597689432551 0.052970881705554015
10 5 0.6886595310907238 0.04860660023185316
10 6 0.6739806320081548 0.04741842666107704
10 7 0.6730377166156982 0.04970359826246403
10 8 0.6721287801563032 0.05126575466870997
10 9 0.6730462113489637 0.05227182122570788
10 10 0.6693764865783214 0.058451671964495536
10 11 0.663888888888889 0.05506503446600879
10 12 0.6666496772001359 0.05941439923287854
10 13 0.6629799524294937 0.058249745612893866
10 14 0.6620625212368332 0.05674412896519435
10 15 0.6620625212368331 0.055240928425989516
10 16 0.6648063200815495 0.0543087431396132
10 17 0.6593017329255861 0.05492261301930248
10 18 0.6583673122663948 0.0576315416782045
10 19 0.6556150186884132 0.05826136760858505
11 1 0.7198521916411824 0.05355443195530491
11 2 0.7051647978253482 0.055265259190053435
11 3 0.7033554196398233 0.05265655477332405
11 4 0.697842337750

19 14 0.662962962962963 0.05704828100017617
19 15 0.6657152565409447 0.05343122128748275
19 16 0.6684760448521917 0.053425052383372654
19 17 0.6666411824668705 0.054443752821485614
19 18 0.6602021746517159 0.060936132300456566
19 19 0.662037037037037 0.06215002819959218


In [126]:
from sklearn.preprocessing import MinMaxScaler

In [228]:
scaling = MinMaxScaler(feature_range=(-1,1)).fit(trainData)
trainDataScaled = scaling.transform(trainData)
testDataScaled = scaling.transform(testData)

C:\Users\Raghav\Anaconda3\lib\site-packages\sklearn\utils\validation.py:429: DataConversionWarning: Data with input dtype object was converted to float64 by MinMaxScaler.
  warnings.warn(msg, _DataConversionWarning)


In [296]:
clfSVM = SVC(kernel='rbf', probability=True, C = 14, gamma = 1)

In [297]:
clfSVM.fit(trainDataScaled, trainLabels) 

SVC(C=14, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma=1, kernel='rbf',
  max_iter=-1, probability=True, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [298]:
clfSVM_probs = clfSVM.predict_proba(testDataScaled)

In [300]:
np.sum(np.max(clfSVM_probs, axis = 1) < 0.6)

42

In [ ]:
# np.sum(clfSVM.predict(testData) != clfRF.predict(testData))

## Naive Bayes

In [209]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB

In [214]:
kf = KFold(n_splits = 10)

for i in range(20):
        
    clfNB = MultinomialNB(alpha = i)
            
    accuracy_lst = []
    
    for train_index, test_index in kf.split(data):
    
        data_train = data.iloc[train_index]
        data_test = data.iloc[test_index]
        
        labels_train = labels[train_index]
        labels_test = labels[test_index]
        
        # ---------------------------------------- #
        
        countVect = CountVectorizer()
        TF_IDF_Transformer = TfidfTransformer()
        
        trainDataCounts = countVect.fit_transform(data_train.text)
        trainDataTF_IDF = TF_IDF_Transformer.fit_transform(trainDataCounts)

        clfNB.fit(trainDataTF_IDF, labels_train)
        
        # ---------------------------------------- #
                        
        testDataCounts = countVect.transform(data_test.text)
        
        testDataTF_IDF = TF_IDF_Transformer.transform(testDataCounts)
        
        # ---------------------------------------- #
                               
        accuracy = np.sum(clfNB.predict(testDataTF_IDF) == labels_test) * 1.0/len(data_test)
        accuracy_lst.append(accuracy) 
        
    print i, np.mean(accuracy_lst), np.std(accuracy_lst)

C:\Users\Raghav\Anaconda3\lib\site-packages\sklearn\naive_bayes.py:699: RuntimeWarning: divide by zero encountered in log
  self.feature_log_prob_ = (np.log(smoothed_fc) -


0 0.4699999999999999 0.05467073155618909
1 0.55 0.11948965552623277
2 0.5566666666666666 0.1653615567308328
3 0.56 0.16983652270475993
4 0.5633333333333334 0.17220788470785986
5 0.5700000000000001 0.1702612371882951
6 0.5766666666666667 0.17515072873892867
7 0.5766666666666667 0.17515072873892867
8 0.5766666666666667 0.17515072873892867
9 0.5766666666666667 0.17515072873892867
10 0.5766666666666667 0.17515072873892867
11 0.5766666666666667 0.17515072873892867
12 0.5766666666666667 0.17515072873892867
13 0.5766666666666667 0.17515072873892867
14 0.5766666666666667 0.17515072873892867
15 0.5766666666666667 0.17515072873892867
16 0.5766666666666667 0.17515072873892867
17 0.5766666666666667 0.17515072873892867
18 0.5766666666666667 0.17515072873892867
19 0.5766666666666667 0.17515072873892867


In [216]:
countVect = CountVectorizer()
TF_IDF_Transformer = TfidfTransformer()

In [217]:
trainDataText = pd.read_csv('train.csv').text

In [218]:
trainDataCounts = countVect.fit_transform(trainDataText)
trainDataTF_IDF = TF_IDF_Transformer.fit_transform(trainDataCounts)

In [219]:
clfNB = MultinomialNB(alpha = 6)
clfNB.fit(trainDataTF_IDF, trainLabels)

MultinomialNB(alpha=6, class_prior=None, fit_prior=True)

In [224]:
testDataText = pd.read_csv('test.csv').text

In [225]:
testDataCounts = countVect.transform(testDataText)
testDataTF_IDF = TF_IDF_Transformer.transform(testDataCounts)
clfNB_probs = clfNB.predict_proba(testDataTF_IDF)

In [227]:
np.sum(np.max(clfNB_probs, axis = 1) < 0.6)

33

In [70]:
# np.sum(clfNB.predict(testDataTF_IDF) == clfRF.predict(testData))

217

## Logistic Regression

In [71]:
from sklearn.linear_model import LogisticRegression

In [307]:
kf = KFold(n_splits = 10)
trainData = np.array(trainData)

for i in range(-6, 6):
    
    for j in ['l1', 'l2']:
        
        clfLR = LogisticRegression(C = 10 ** i, penalty = j)

        accuracy_lst = []

        for train_index, test_index in kf.split(trainData):

            data_train = trainData[train_index]
            data_test = trainData[test_index]

            labels_train = labels[train_index]
            labels_test = labels[test_index]
            
            clfLR.fit(data_train, labels_train) 

            accuracy = np.sum(clfLR.predict(data_test) == labels_test) * 1.0/len(data_test)
            accuracy_lst.append(accuracy) 

        print 10 ** i, j, np.mean(accuracy_lst), np.std(accuracy_lst)

1e-06 l1 0.4316173972137275 0.08261826648208546
1e-06 l2 0.5683826027862725 0.08261826648208546
1e-05 l1 0.4316173972137275 0.08261826648208546
1e-05 l2 0.5647043832823649 0.08236471690431786
0.0001 l1 0.5683826027862725 0.08261826648208546
0.0001 l2 0.6436034658511723 0.0828396636286068
0.001 l1 0.5665392456676861 0.08385288897080138
0.001 l2 0.6683995922528032 0.09603079509316109
0.01 l1 0.6693255181787292 0.09309754894790312
0.01 l2 0.6785083248386001 0.08739867903274938
0.1 l1 0.6766989466530752 0.07751906949070099
0.1 l2 0.6886340468909277 0.06787354818443965
1 l1 0.6932296975874956 0.06318297917562715
1 l2 0.6923122663948351 0.06278368167427573
10 l1 0.6968909276248727 0.0606571281101051
10 l2 0.6959734964322122 0.060157071662847866
100 l1 0.6968909276248727 0.0606571281101051
100 l2 0.6968909276248727 0.0606571281101051
1000 l1 0.6968909276248727 0.0606571281101051
1000 l2 0.6968909276248727 0.0606571281101051
10000 l1 0.6968909276248727 0.0606571281101051
10000 l2 0.69689092762

In [308]:
clfLR = LogisticRegression(C = 10, penalty = 'l1')

In [309]:
clfLR.fit(trainData, trainLabels)

LogisticRegression(C=10, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l1', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [310]:
clfLR_probs = clfLR.predict_proba(testData)

In [311]:
np.sum(np.max(clfLR_probs, axis = 1) < 0.60)

86

## Neural Network

In [76]:
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler  

In [77]:
scaler = StandardScaler()  
# Don't cheat - fit only on training data
scaler.fit(trainData)  
trainDataScaled = scaler.transform(trainData)  
# apply same transformation to test data
testDataScaled = scaler.transform(testData)  

In [84]:
clfNN = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(100, 5))
clfNN.fit(trainDataScaled, trainLabels)                         

MLPClassifier(activation='relu', alpha=1e-05, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100, 5), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='lbfgs', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)

In [85]:
clfNN_probs = clfNN.predict_proba(testDataScaled)

In [86]:
np.sum(np.max(clfNN_probs, axis = 1) < 0.60)

9

In [99]:
np.sum(np.max(clfNN_probs, axis = 1) == 1.0)

24

## Ensemble Methods

In [314]:
predRF = np.max(clfRF_probs, axis = 1) * np.sign(np.argmax(clfRF_probs, axis = 1) - 0.5)

In [315]:
predSVM = np.max(clfSVM_probs, axis = 1) * np.sign(np.argmax(clfSVM_probs, axis = 1) - 0.5)

In [316]:
predNB = np.max(predicted, axis = 1) * np.sign(np.argmax(predicted, axis = 1) - 0.5)

In [317]:
predLR = np.max(clfLR_probs, axis = 1) * np.sign(np.argmax(clfLR_probs, axis = 1) - 0.5)

In [320]:
# predNN = np.max(clfNN_probs, axis = 1) * np.sign(np.argmax(clfNN_probs, axis = 1) - 0.5)

In [133]:
eps = 0.001

In [322]:
weightRF = np.log(np.max(clfRF_probs, axis = 1) / (1 - np.max(clfRF_probs, axis = 1)))
# weightRF

In [323]:
weightSVM = np.log(np.max(clfSVM_probs, axis = 1) / (1 - np.max(clfSVM_probs, axis = 1)))
# weightSVM

In [324]:
weightNB = np.log(np.max(predicted, axis = 1) / (1 - np.max(predicted, axis = 1)))
# weightNB

In [325]:
weightLR = np.log(np.max(clfLR_probs, axis = 1) / (1 - np.max(clfLR_probs, axis = 1)))
# weightLR

In [137]:
weightNN = np.log(np.max(clfNN_probs, axis = 1) / (1 - np.max(clfNN_probs, axis = 1)))
# weightNN

In [346]:
weightedScore = weightNB * clfNB.predict(testDataTF_IDF) + weightRF * clfRF.predict(testData) + weightSVM * clfRF.predict(testDataScaled)

In [348]:
predFinalWeighted = np.sign(weightedScore)

In [349]:
pd.DataFrame(predFinalWeighted.astype('int')).to_csv('NEW_NB_RF_SVM.csv', header=['Label'], index_label = ['ID'])

In [335]:
np.sum(clfRF.predict(testData) != clfSVM.predict(testDataScaled))

53

In [336]:
np.sum(clfNB.predict(testDataTF_IDF) != clfSVM.predict(testDataScaled))

106

In [337]:
np.sum(clfRF.predict(testData) != clfNB.predict(testDataTF_IDF))

89

In [345]:
np.sum(abs(clfRF.predict(testData) + clfNB.predict(testDataTF_IDF) + clfSVM.predict(testDataScaled)) == 3)

176

In [350]:
probWeightedScore = weightNB * clfNB.predict(testDataTF_IDF) * np.max(clfNB_probs, axis = 1) + weightRF * clfRF.predict(testData) * np.max(clfRF_probs, axis = 1) + weightSVM * clfSVM.predict(testDataScaled) * np.max(clfSVM_probs, axis = 1)

In [351]:
predFinalProbWeighted = np.sign(probWeightedScore)

In [355]:
np.sum(predFinalProbWeighted != predFinalWeighted)

38

In [356]:
pd.DataFrame(predFinalProbWeighted.astype('int')).to_csv('NEW_NB_RF_SVM_PW.csv', header=['Label'], index_label = ['ID'])

### Stacked Model

In [373]:
stackedModel = pd.DataFrame({'RF': clfRF.predict_proba(trainData)[:, 0], 'NB': clfNB.predict_proba(trainDataTF_IDF)[:, 0], 'SVM': clfSVM.predict_proba(trainDataScaled)[:, 0]})

In [377]:
kf = KFold(n_splits = 10)
stackedModel = np.array(stackedModel)

for i in range(-2, 6):
    
    for j in ['l1', 'l2']:
        
        clfStack = LogisticRegression(C = 10 ** i, penalty = j)

        accuracy_lst = []

        for train_index, test_index in kf.split(stackedModel):

            data_train = stackedModel[train_index]
            data_test = stackedModel[test_index]

            labels_train = labels[train_index]
            labels_test = labels[test_index]
            
            clfStack.fit(data_train, labels_train) 

            accuracy = np.sum(clfStack.predict(data_test) == labels_test) * 1.0/len(data_test)
            accuracy_lst.append(accuracy) 

        print 10 ** i, j, np.mean(accuracy_lst), np.std(accuracy_lst)

0.01 l1 0.977055725450221 0.013752994144232221
0.01 l2 0.9935694869181109 0.008259751140237595
0.1 l1 1.0 0.0
0.1 l2 1.0 0.0
1 l1 1.0 0.0
1 l2 1.0 0.0
10 l1 1.0 0.0
10 l2 1.0 0.0
100 l1 1.0 0.0
100 l2 1.0 0.0
1000 l1 1.0 0.0
1000 l2 1.0 0.0
10000 l1 1.0 0.0
10000 l2 1.0 0.0
100000 l1 1.0 0.0
100000 l2 1.0 0.0


In [378]:
clfStack = LogisticRegression(C = 1, penalty = 'l1')
clfStack.fit(stackedModel, trainLabels)

LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l1', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [379]:
stackedModelPredict = pd.DataFrame({'RF': clfRF.predict_proba(testData)[:, 0], 'NB':  clfNB.predict_proba(testDataTF_IDF)[:, 0], 'SVM': clfSVM.predict_proba(testDataScaled)[:, 0]})

In [381]:
predStack = clfStack.predict(stackedModelPredict)

In [383]:
pd.DataFrame(predStack.astype('int')).to_csv('NEW PRED STACK.csv', header=['Label'], index_label = ['ID'])